In [ ]:
import iris
import iris.plot as iplt
import numpy as np
from iris.coords import DimCoord
from iris.coord_systems import TransverseMercator,GeogCS
from iris.cube import Cube
from cf_units import Unit
import cf_units
import os
import glob
from pyproj import Proj, transform
import sys
import warnings
import multiprocessing as mp
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

ems_hourly = ['01', '04', '06', '07', '08', '09', '10', '11', '12', '13', '15']
ems_30mins = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc013',  'bc015',  'bc016', 'bc017', 'bc018', 'bc012']
yrs_range = '2002_2020'

def mask_cube(cube, gb_mask):
    masked_cube_data = cube * gb_mask[np.newaxis, :, :]

    # APPLY THE MASK
    reshaped_mask = np.tile(gb_mask, (cube.shape[0], 1, 1))
    reshaped_mask = reshaped_mask.astype(int)
    reversed_array = ~reshaped_mask.astype(bool)

    # Mask the cube
    masked_cube = iris.util.mask_cube(cube, reversed_array)
    
    return masked_cube

### Get the mask
gb_mask_2km = np.load("/nfs/a319/gy17m2a/PhD/datadir/Masks/UKCP18_2.2km_GB_Mask.npy")
gb_mask_2km_bng = np.load("/nfs/a319/gy17m2a/PhD/datadir/Masks/UKCP18_2.2km_bng_GB_Mask.npy")
gb_mask_12km_wgs84 = np.load("/nfs/a319/gy17m2a/PhD/datadir/Masks/UKCP18_12km_wgs84_GB_Mask.npy")
gb_mask_12km = np.load("/nfs/a319/gy17m2a/PhD/datadir/Masks/UKCP18_12km_GB_Mask.npy")
gb_mask_nimrod = np.load('/nfs/a319/gy17m2a/PhD/datadir/Masks/nimrod_1km_GB_Mask_[1100,200].npy')
# gb_mask_12km_bng = np.load("/nfs/a319/gy17m2a/PhD/datadir/UKCP18_12km_bng_GB_Mask.npy")

In [ ]:
root_fp = "/nfs/a319/gy17m2a/"
os.chdir(root_fp)

# Create path to files containing functions
sys.path.insert(0, root_fp + 'PhD/Scripts/DataProcessing/Regridding')
from Regridding_functions import *
# Create path to files containing functions
sys.path.insert(0, root_fp + 'Scripts/GlobalFunctions')
from Spatial_plotting_functions import *
from Spatial_geometry_functions import *

gb_gdf = create_gb_outline({'init' :'epsg:3857'})

# Load UKCP18 12km model data to use in regriddding
file_model_12km=f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/12km/01/pr_rcp85_land-rcm_uk_12km_01_day_19801201-19901130.nc'
cube_12km=iris.load_cube(file_model_12km)

file_model_2_2km ='/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_original/01/1980_2001/pr_rcp85_land-cpm_uk_2.2km_01_1hr_19910601-19910630.nc'
cube_2km =iris.load_cube(file_model_2_2km)

file_model_2_2km_bng ='/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_bng/01/1980_2001/bng_pr_rcp85_land-cpm_uk_2.2km_01_1hr_19910601-19910630.nc'
cube_2km_bng =iris.load_cube(file_model_2_2km_bng)

file_model_2_2km_bng_30mins = '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_original/bc005/2002_2020/bc005a.pr200101.nc'
file_model_2_2km_bng_30mins = '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_original/bc005/2002_2020/bc005a.pr200101.nc'
cube_2km_30mins = iris.load_cube(file_model_2_2km_bng_30mins)
# # Trim ensemble member dimension
cube_2km_30mins = trim_to_bbox_of_region_regriddedobs(cube_2km_30mins, gb_gdf)
# # Transform to BNG
cube_2km_30mins_bng, lats_bng, lons_bng = convert_rotatedpol_to_bng(cube_2km_30mins.copy())

# remove ensemble member dimension
cube_2km = cube_2km[0,:,:,:]
cube_12km = cube_12km[0,:,:,:]

in_jja=iris.Constraint(time=lambda cell: 6 <= cell.point.month <= 8)

In [ ]:
lsm = iris.load("/nfs/a319/gy17m2a/PhD/datadir/Masks/lsm_land-cpm_BI_5km.nc")[0]
lsm_2km = lsm.regrid(cube_2km_bng, iris.analysis.AreaWeighted(mdtol=0.9)) 
lsm_2km = lsm.regrid(cube_2km_bng, iris.analysis.Nearest()) 

broadcasted_lsm_2km_data = np.broadcast_to(lsm_2km.data.data, cube_2km_bng.shape)
broadcasted_lsm_2km_int = broadcasted_lsm_2km_data.astype(int)
broadcasted_lsm_2km_data_reversed = ~broadcasted_lsm_2km_data.astype(bool)

broadcasted_lsm_2km_30mins_data = np.broadcast_to(lsm_2km.data.data, cube_2km_30mins_bng.shape)
broadcasted_lsm_2km_30mins_data_reversed = ~broadcasted_lsm_2km_30mins_data.astype(bool)

## Regrid 30 mins UKCP18 data to 12km - DONE
For this need to use version of the regridding cube which is trimmed to the leeds-at-centre region, as the 30 mins data is already clipped to this extent

In [5]:
# for em in ems_30mins:
#     print(em)
#     os.chdir(f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_original/{em}/2002_2020/")
#     # establish paths to directories
#     output_fp_bng = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{em}/{yrs_range}/"
#     output_fp_bng_masked = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng_masked/{em}/{yrs_range}/"
#     output_fp_bng_regridded_12km = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng_regridded_12km_masked/{em}/AreaWeighted/{yrs_range}/"

#     # create the directories
#     if not os.path.isdir(output_fp_bng):
#         os.makedirs(output_fp_bng)
#     if not os.path.isdir(output_fp_bng_masked):
#         os.makedirs(output_fp_bng_masked)    
#     if not os.path.isdir(output_fp_bng_regridded_12km):
#         os.makedirs(output_fp_bng_regridded_12km)    
            
#     # loop through the files
#     for filename in np.sort(glob.glob("*")): 
#         if '2000' not in filename and '2020' not in filename:
#             print(filename)
#             if not os.path.isfile(output_fp_bng +  f"bng_{filename}"):
#                 print("creating")

#                 # Load the data
#                 cube_2km = iris.load(filename)[0]
#                 # Trim
#                 cube_2km = trim_to_bbox_of_region_regriddedobs(cube_2km, gb_gdf)
#                 # Transform to BNG
#                 cube_2km_bng, lats_bng, lons_bng = convert_rotatedpol_to_bng(cube_2km.copy())
#                 # Mask to GB
#                 cube_2km_bng_masked = iris.util.mask_cube(cube_2km_bng.copy(), broadcasted_lsm_2km_30mins_data_reversed)
#                 # Regrid to 12km
#                 cube_2km_bng_masked_regridded_12km = cube_2km_bng_masked.regrid(cube_12km, iris.analysis.AreaWeighted(mdtol=0.8)) 
#                 # Save 
#                 iris.save(cube_2km_bng, output_fp_bng +  f"bng_{filename}")     
#                 iris.save(cube_2km_bng_masked, output_fp_bng_masked +  f"bng_{filename}")
#                 iris.save(cube_2km_bng_masked_regridded_12km, output_fp_bng_regridded_12km +  f"bng_rg_{filename}") 
            
#             else:
#                 print("already exists")    

bc005
bc005a.pr200101.nc
already exists
bc005a.pr200102.nc
already exists
bc005a.pr200103.nc
already exists
bc005a.pr200104.nc
already exists
bc005a.pr200105.nc
already exists
bc005a.pr200106.nc
already exists
bc005a.pr200107.nc
already exists
bc005a.pr200108.nc
already exists
bc005a.pr200109.nc
already exists
bc005a.pr200110.nc
already exists
bc005a.pr200111.nc
already exists
bc005a.pr200112.nc
already exists
bc005a.pr200201.nc
already exists
bc005a.pr200202.nc
already exists
bc005a.pr200203.nc
already exists
bc005a.pr200204.nc
already exists
bc005a.pr200205.nc
already exists
bc005a.pr200206.nc
already exists
bc005a.pr200207.nc
already exists
bc005a.pr200208.nc
already exists
bc005a.pr200209.nc
already exists
bc005a.pr200210.nc
already exists
bc005a.pr200211.nc
already exists
bc005a.pr200212.nc
already exists
bc005a.pr200301.nc
already exists
bc005a.pr200302.nc
already exists
bc005a.pr200303.nc
already exists
bc005a.pr200304.nc
already exists
bc005a.pr200305.nc
already exists
bc005a.p

bc006a.pr200908.nc
already exists
bc006a.pr200909.nc
already exists
bc006a.pr200910.nc
already exists
bc006a.pr200911.nc
already exists
bc006a.pr200912.nc
already exists
bc006a.pr201001.nc
already exists
bc006a.pr201002.nc
already exists
bc006a.pr201003.nc
already exists
bc006a.pr201004.nc
already exists
bc006a.pr201005.nc
already exists
bc006a.pr201006.nc
already exists
bc006a.pr201007.nc
already exists
bc006a.pr201008.nc
already exists
bc006a.pr201009.nc
already exists
bc006a.pr201010.nc
already exists
bc006a.pr201011.nc
already exists
bc006a.pr201012.nc
already exists
bc006a.pr201101.nc
already exists
bc006a.pr201102.nc
already exists
bc006a.pr201103.nc
already exists
bc006a.pr201104.nc
already exists
bc006a.pr201105.nc
already exists
bc006a.pr201106.nc
already exists
bc006a.pr201107.nc
already exists
bc006a.pr201108.nc
already exists
bc006a.pr201109.nc
already exists
bc006a.pr201110.nc
already exists
bc006a.pr201111.nc
already exists
bc006a.pr201112.nc
already exists
bc006a.pr20120

already exists
bc007a.pr201212.nc
already exists
bc007a.pr201301.nc
already exists
bc007a.pr201302.nc
already exists
bc007a.pr201303.nc
already exists
bc007a.pr201304.nc
already exists
bc007a.pr201305.nc
already exists
bc007a.pr201306.nc
already exists
bc007a.pr201307.nc
already exists
bc007a.pr201308.nc
already exists
bc007a.pr201309.nc
already exists
bc007a.pr201310.nc
already exists
bc007a.pr201311.nc
already exists
bc007a.pr201312.nc
already exists
bc007a.pr201401.nc
already exists
bc007a.pr201402.nc
already exists
bc007a.pr201403.nc
already exists
bc007a.pr201404.nc
already exists
bc007a.pr201405.nc
already exists
bc007a.pr201406.nc
already exists
bc007a.pr201407.nc
already exists
bc007a.pr201408.nc
already exists
bc007a.pr201409.nc
already exists
bc007a.pr201410.nc
already exists
bc007a.pr201411.nc
already exists
bc007a.pr201412.nc
already exists
bc007a.pr201501.nc
already exists
bc007a.pr201502.nc
already exists
bc007a.pr201503.nc
already exists
bc007a.pr201504.nc
already exists

already exists
bc009a.pr201711.nc
already exists
bc009a.pr201712.nc
already exists
bc009a.pr201801.nc
already exists
bc009a.pr201802.nc
already exists
bc009a.pr201803.nc
already exists
bc009a.pr201804.nc
already exists
bc009a.pr201805.nc
already exists
bc009a.pr201806.nc
already exists
bc009a.pr201807.nc
already exists
bc009a.pr201808.nc
already exists
bc009a.pr201809.nc
already exists
bc009a.pr201810.nc
already exists
bc009a.pr201811.nc
already exists
bc009a.pr201812.nc
already exists
bc009a.pr201901.nc
already exists
bc009a.pr201902.nc
already exists
bc009a.pr201903.nc
already exists
bc009a.pr201904.nc
already exists
bc009a.pr201905.nc
already exists
bc009a.pr201906.nc
already exists
bc009a.pr201907.nc
already exists
bc009a.pr201908.nc
already exists
bc009a.pr201909.nc
already exists
bc009a.pr201910.nc
already exists
bc009a.pr201911.nc
already exists
bc009a.pr201912.nc
already exists
bc010
bc010a.pr200101.nc
already exists
bc010a.pr200102.nc
already exists
bc010a.pr200103.nc
already 

already exists
bc010a.pr201901.nc
already exists
bc010a.pr201902.nc
already exists
bc010a.pr201903.nc
already exists
bc010a.pr201904.nc
already exists
bc010a.pr201905.nc
already exists
bc010a.pr201906.nc
already exists
bc010a.pr201907.nc
already exists
bc010a.pr201908.nc
already exists
bc010a.pr201909.nc
already exists
bc010a.pr201910.nc
already exists
bc010a.pr201911.nc
already exists
bc010a.pr201912.nc
already exists
bc011
bc011a.pr200101.nc
creating
bc011a.pr200102.nc
creating
bc011a.pr200103.nc
creating
bc011a.pr200104.nc
creating
bc011a.pr200105.nc
creating
bc011a.pr200106.nc
creating
bc011a.pr200107.nc
creating
bc011a.pr200108.nc
creating
bc011a.pr200109.nc
creating
bc011a.pr200110.nc
creating


KeyboardInterrupt: 

## Regrid NIMROD - DONE

In [ ]:
file_nimrod_1km =f"/nfs/a161/gy17m2a/PhD/datadir/NIMROD/30mins/OriginalFormat_1km/Unfiltered/2018/metoffice-c-band-rain-radar_uk_20180601_30mins.nc"
nimrod_1km =iris.load_cube(file_nimrod_1km)
nimrod_1km= trim_to_bbox_of_region_obs(nimrod_1km, gb_gdf, 'projection_y_coordinate', 'projection_x_coordinate')

lsm_1km = lsm.regrid(nimrod_1km, iris.analysis.Nearest()) 

In [ ]:
for year in range(2006,2021):
    print(year)
    # Change directory to be for correct year
    os.chdir(f"/nfs/a161/gy17m2a/PhD/datadir/NIMROD/30mins/OriginalFormat_1km/Filtered_100/{year}")
    # Define filepaths to save files to
    output_dir_2_2km = f"/nfs/a161/gy17m2a/PhD/datadir/NIMROD/30mins/NIMROD_regridded_2.2km/Filtered_100/AreaWeighted/{year}/"
    # Create these directories if they don't exist already
    if not os.path.isdir(output_dir_2_2km):
        os.makedirs(output_dir_2_2km)
    # Loop through all the diles in the 1km folder    
    for filename in sorted(glob.glob("*")):
        if filename[35:37] not in ['06','07','08']:
            print(filename)

            # Create version of filename specifying it is regridded
            filename_to_save_to = f"rg_{filename}"
            if not os.path.isfile(output_dir_2_2km + filename_to_save_to):
                print("Doesn't exist: Creating now")
                cube = iris.load(filename)[0] 
                # Fill in missing bounds
                cube.coord('projection_y_coordinate').guess_bounds()
                cube.coord('projection_x_coordinate').guess_bounds()
                # Align small rounding error in coordinates
                cube.coord('projection_x_coordinate').coord_system = cube_2km_bng.coord('projection_x_coordinate').coord_system
                cube.coord('projection_y_coordinate').coord_system = cube_2km_bng.coord('projection_y_coordinate').coord_system

                # Trim 
                cube= trim_to_bbox_of_region_obs(cube, gb_gdf, 'projection_y_coordinate', 'projection_x_coordinate')

                # Reshape to match the data (data not always same number of timeslices so eneds redoing each time)
                broadcasted_lsm_1km_data = np.broadcast_to(lsm_1km.data.data, cube.shape)
                broadcasted_lsm_1km_data_reversed = ~broadcasted_lsm_1km_data.astype(bool)

                # Apply mask
                cube_masked = iris.util.mask_cube(cube.copy(), broadcasted_lsm_1km_data_reversed)

                # Area Weighted
                reg_cube_masked =cube_masked.regrid(cube_2km_bng,iris.analysis.AreaWeighted())    
                print("Regridded")
                # Save 
                iris.save(reg_cube_masked, output_dir_2_2km + filename_to_save_to)    
            else:
                print("file existss")

In [ ]:
# fig, ax=plt.subplots(figsize=(20,20))
# iplt.contourf(cube_masked[1,:,:])
# plt.gca().coastlines()

## Regrid CEH-GEAR

### Test method with one file

In [10]:
### Read in reformatted data and set coordinate bounds
# (Might have deleted bit of code from previous versions that do the reformatting, check git history)

cube_1km = iris.load('/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_reformatted/rf_CEH-GEAR-1hr_199006.nc', in_jja)[0]
cube_1km.coord('projection_y_coordinate').guess_bounds()
cube_1km.coord('projection_x_coordinate').guess_bounds()

### Regrid to 12km using different available methods
cube_aw= cube_1km.regrid(cube_2km_bng, iris.analysis.AreaWeighted()) 
cube_nn= cube_1km.regrid(cube_2km_bng, iris.analysis.Nearest()) 
cube_l= cube_1km.regrid(cube_2km_bng, iris.analysis.Linear()) 

In [18]:
# fig, ax= plt.subplots(figsize=(15,15))
# iplt.contourf(cube_1km[15,:,:])
# plt.gca().coastlines(resolution='10m', color='black', linewidth=0.5);

### Run properly on all files

In [129]:
# contraint so we only bother for JJA data
in_jja=iris.Constraint(time=lambda cell: 6 <= cell.point.month <= 8)
# Directory where data will be saved 
dir_to_save_to = f"/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_regridded_12km/AreaWeighted/"
# For all files in the directory
for filename in np.sort(glob.glob("/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_reformatted/*")):
    print(filename)
    filename_to_save_to = f"rg_{filename}"
    # Check if it doesn't already exist
    if not os.path.isfile(dir_to_save_to + filename_to_save_to):
        print("Creating")
        # if it's in JJA
        try:
            cube_1km = iris.load(filename, in_jja)[0] 
            print("JJA")
            # Coordinates dont have bounds
            cube_1km.coord('projection_y_coordinate').guess_bounds()
            cube_1km.coord('projection_x_coordinate').guess_bounds()
            # Regrid with area weighted
            cube_aw= cube_1km.regrid(cube_12km, iris.analysis.AreaWeighted()) 
            # File path to save to
            dir_to_save_to = f"/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_regridded_12km/AreaWeighted/"
            filename_to_save_to = f"rg_{filename}"
            iris.save(cube_aw, dir_to_save_to + filename_to_save_to)
        except:
            print("Not JJA")
    else:
        print("already exists")

rf_CEH-GEAR-1hr_199001.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199002.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199003.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199004.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199005.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199006.nc
Creating
JJA
rf_CEH-GEAR-1hr_199007.nc
already exists
rf_CEH-GEAR-1hr_199008.nc
Creating
JJA
rf_CEH-GEAR-1hr_199009.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199010.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199011.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199012.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199101.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199102.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199103.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199104.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199105.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199106.nc
already exists
rf_CEH-GEAR-1hr_199107.nc
Creating
JJA
rf_CEH-GEAR-1hr_199108.nc
Creating
JJA
rf_CEH-GEAR-1hr_199109.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199110.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199111.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199112.nc
Creat

Exception ignored in: <function CFReader.__del__ at 0x7f64deb683b0>
Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/fileformats/cf.py", line 1263, in __del__
    self._dataset.close()
AttributeError: 'CFReader' object has no attribute '_dataset'


rf_CEH-GEAR-1hr_199908.nc
Creating
JJA
rf_CEH-GEAR-1hr_199909.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199910.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199911.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199912.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200001.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200002.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200003.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200004.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200005.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200006.nc
Creating
JJA
rf_CEH-GEAR-1hr_200007.nc
Creating
JJA
rf_CEH-GEAR-1hr_200008.nc
Creating
JJA
rf_CEH-GEAR-1hr_200009.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200010.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200011.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200012.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200101.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200102.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200103.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200104.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200105.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200106.nc
Creating
JJA
rf_CEH-GEAR-1hr_200107.nc
Creat